# Co-factor swap

<!-- The balance of co-factors within the cells is important 
In the implementated pathway of progesterone nine NADP and one NAD molecules 
In order for the cell to produce one pregesterone molecule through the implementated pathway nine NADPH molecules and one NAD $^+$ molecules are needed. 
 -->



The balance of co-factors within a cell is important to obtain a high theoretical yield of a given product (King, Zachary A., and Adam M. Feist, 2014). The NADP(H) and NAD(H) co-factors are needed in order for a cell to produce progesterone through the implemented pathway. Therefore, by increasing the concentration of available NADP(H) and NAD(H) co-factors in the cell, it should be theoretically possible to increase the yield of progesterone. 
This can be done using the “CofactorSwapOptimization” algorithm where possible target reactions to swap co-factors can be found.
The obvious choice would be to increase available NAD(H) and NADP(H) on the cost of FAD(H2). However, no FADH2 in the cytosol is included in the model which makes this approach impossible. Therefore, we instead used the algorithm to increase the concentration of available NADP(H) on the cost of NAD(H). Since nine NADP(H) and only one NAD(H) are used in the implemented pathway, this approach should also theoretically increase the yield of progesterone.

In the code the following was done:
- Model iMM904_progesterone was loaded
- A demand for progesterone was made 
- The objective was set to maximize the production of progesterone
- The theoretical yield was set to production of progesterone divided by the optake of glucose
- The algorithm was run and the result was visualized

In [133]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [134]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [135]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [136]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [137]:
# Setting the progesterone yield as the formation of progesterone divided by the optake of glucose
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [138]:
# Running the CofactorSwapOptimization Algorithm
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nadp_c","nadph_c"],["nad_c","nadh_c"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap._metadata["swap_pairs"][0][0]} and {swap._metadata["swap_pairs"][0][1]} on the cost of {swap._metadata["swap_pairs"][1][0]} and {swap._metadata["swap_pairs"][1][1]}')
solution_1 = swap.run(max_size = 10)

Co-factor swap optimization of the model iMM904_progesterone used to increase available nadp_c and nadph_c on the cost of nad_c and nadh_c
Starting optimization at Wed, 23 Nov 2022 09:14:13


HBox()

Finished after 00:03:57


In [ ]:
solution_1

In [139]:
# Printing result
print("The following reactions can be used as targets for co-factor swapping:\n")
target_set = set()
for target in solution_1.data_frame.targets:
    for id in target:
        target_set.add(id)
for id in target_set:
    print(model.reactions.get_by_id(id))

The following reactions can be used as targets for co-factor swapping:

ALCD2x_copy1: etoh_c + nad_c --> acald_c + h_c + nadh_c
ALCD2x_copy2: etoh_c + nad_c <=> acald_c + h_c + nadh_c
MDH: mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
HMGCOAR: coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c
GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c


The reactions listed above can be used as targets for co-factor swapping to increase the theoretical yield of progesterone. From the 

In [140]:
# Loading model
infilename = 'models/iMM904.xml'
print(f"Loading {infilename}")
model = read_sbml_model(infilename)

Loading models/iMM904.xml


In [141]:
# Add all new metabolites from metabolites.csv
new_metabolites = dict()
with open("data/metabolites.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        m = Metabolite(
            line[0],
            formula=line[1],
            name=line[2],
            compartment=line[3])
        print(f'Adding metabolite {m.name} (id: {m})')
        new_metabolites[line[0]] = m

Adding metabolite cholesterol (id: cholesterol_c)
Adding metabolite 7-dehydrocholesterol (id: dehydrocholesterol_c)
Adding metabolite pregnenolone (id: pregnenolone_c)
Adding metabolite progesterone (id: progesterone_c)
Adding metabolite 5alpha-Cholesta-7_24-dien-3beta-ol (id: cholesta724dien3betaol_c)
Adding metabolite 5alpha-Cholest-8-en-3beta-ol (id: cholesta8en3betaol_c)
Adding metabolite lathosterol (id: lathosterol_c)
Adding metabolite 4-methylpentanal (id: methylpentanal_c)


In [142]:
# Make co-factor swap with DHCR7 all new reactions from reactions.csv
with open("data/co-factor_swap_reactions.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        if line[3] != "":
            r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        if line[1] != "":
            r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                if metaboliteID in model.metabolites:
                    metaboliteID = model.metabolites.get_by_id(metaboliteID)
                else:
                    metaboliteID = new_metabolites[metaboliteID]
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model.add_reactions([r])

model.reactions.CHLSTI.gpr = GPR.from_string("EBP")
print(f'Adding gene annotation (EBP) to reaction: {model.reactions.CHLSTI}')

model.add_boundary(model.metabolites.get_by_id("methylpentanal_c"), type="sink")
print(f'Adding reaction {model.reactions.SK_methylpentanal_c.name}: {model.reactions.SK_methylpentanal_c.reaction}')

Adding reaction R07498: h_c + nadh_c + zymst_c <=> cholesta8en3betaol_c + nad_c | enzyme DHCR24
Adding reaction R05703: cholesta724dien3betaol_c + h_c + nadh_c <=> lathosterol_c + nad_c | enzyme DHCR24
Adding reaction R01456: dehydrocholesterol_c + h_c + nadh_c --> cholesterol_c + nad_c | enzyme DHCR7
Adding reaction ECYP11A1: cholesterol_c + 6.0 h_c + 6.0 nadh_c + 3.0 o2_c --> 4.0 h2o_c + methylpentanal_c + 6.0 nad_c + pregnenolone_c | enzyme CYP11A1
Adding reaction R02216: nadp_c + pregnenolone_c <=> h_c + nadph_c + progesterone_c | enzyme HSD3B
Adding reaction R03353: cholesta8en3betaol_c --> lathosterol_c | enzyme YMR202W
Adding reaction R07215: h_c + lathosterol_c + nadph_c + o2_c --> dehydrocholesterol_c + 2.0 h2o_c + nadp_c | enzyme YLR056W
Adding reaction R04804: zymst_c --> cholesta724dien3betaol_c | enzyme YMR202W
Adding gene annotation (EBP) to reaction: CHLSTI: amet_c + o2_c + zymst_c --> ahcys_c + ergtetrol_c + 2.0 h2o_c + h_c
Adding reaction 4-methylpentanal sink: methylp

In [143]:
# Saving new model
outfilename = "models/iMM904_progesterone_coswap.xml"
model.id = outfilename.split("/")[-1].split(".")[0]
print(f"Saving to {outfilename}")
write_sbml_model(model, outfilename)

Saving to models/iMM904_progesterone_coswap.xml


In [ ]:
# Loading models
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model_progesterone = read_sbml_model(infilename)
infilename = 'models/iMM904_progesterone_coswap.xml'
print(f"Loading {infilename}")
model_progesterone_coswap = read_sbml_model(infilename)

Loading models/iMM904_progesterone.xml
Loading models/iMM904_progesterone_coswap.xml


In [ ]:
# Add demand reaction for progesterone
model_progesterone.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model_progesterone.reactions.DM_progesterone_c.name}: {model_progesterone.reactions.DM_progesterone_c.reaction}')
model_progesterone_coswap.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model_progesterone_coswap.reactions.DM_progesterone_c.name}: {model_progesterone_coswap.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 
Adding reaction progesterone demand: progesterone_c --> 


In [ ]:
print(f'{model_progesterone.id}: µ = {model_progesterone.optimize().objective_value}')
print(f'{model_progesterone_coswap.id}: µ = {model_progesterone_coswap.optimize().objective_value}')

iMM904_progesterone: µ = 0.2878657037040161
iMM904_progesterone_coswap: µ = 0.28786570370401676


In [ ]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model_progesterone.reactions.R02216
model_progesterone.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')
objective_reaction = model_progesterone_coswap.reactions.R02216
model_progesterone_coswap.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c
Objective set to maximize R02216: nadp_c + pregnenolone_c <=> h_c + nadph_c + progesterone_c


In [ ]:
print(f'{model_progesterone.id}: progesterone flux = {model_progesterone.optimize().objective_value}')
print(f'{model_progesterone_coswap.id}: progesterone flux = {model_progesterone_coswap.optimize().objective_value}')

iMM904_progesterone: progesterone flux = 0.14285714285714285
iMM904_progesterone_coswap: progesterone flux = 0.14285714285714288


In [ ]:
print(model_progesterone.genes.DHCR7.reactions)
print(model_progesterone_coswap.genes.DHCR7.reactions)

frozenset({<Reaction R01456 at 0x7f5c33502ee0>})
frozenset({<Reaction R01456 at 0x7f5c2cd19b50>})


In [ ]:
model_progesterone.reactions.R01456

Reaction identifier,R01456
Name,Cholesterol:NAD+ delta7-oxidoreductase
Memory address,0x07f5c33502ee0
Stoichiometry,dehydrocholesterol_c + h_c + nadph_c --> cholesterol_c + nadp_c 7-dehydrocholesterol + H+ + Nicotinamide adenine dinucleotide phosphate - reduced --> cholesterol + Nicotinamide adenine dinucleotide phosphate
GPR,DHCR7
Lower bound,0.0
Upper bound,999999.0


In [ ]:
model_progesterone_coswap.reactions.R01456

Reaction identifier,R01456
Name,Cholesterol:NAD+ delta7-oxidoreductase
Memory address,0x07f5c2cd19b50
Stoichiometry,dehydrocholesterol_c + h_c + nadh_c --> cholesterol_c + nad_c 7-dehydrocholesterol + H+ + Nicotinamide adenine dinucleotide - reduced --> cholesterol + Nicotinamide adenine dinucleotide
GPR,DHCR7
Lower bound,0.0
Upper bound,999999.0


### References
King, Zachary A., and Adam M. Feist. "Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae." Metabolic Engineering 24 (2014): 117-128.